In [1]:
from flask import Flask, request, jsonify
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint
import joblib
import random
import os
import math
import mysql.connector

In [2]:
def set_seed(seed=42):
    import random
    random.seed(seed)
    np.random.seed(seed)
    import tensorflow as tf
    tf.random.set_seed(seed)

set_seed()

def load_data_from_db():
    conn = mysql.connector.connect(
        host="127.0.0.1",
        user="root",
        password="",
        database="db_inventory_web"
    )
    cursor = conn.cursor()
    query = "SELECT id_bakery, date, item_name, quantity FROM bakery_sale"
    cursor.execute(query)
    columns = [col[0] for col in cursor.description]
    data = cursor.fetchall()
    df = pd.DataFrame(data, columns=columns)
    cursor.close()
    conn.close()
    return df

In [3]:
def preprocess_data(df):
    df['date'] = pd.to_datetime(df['date'])
    df['quantity'] = pd.to_numeric(df['quantity'], errors='coerce')
    df = df.dropna()  # Drop rows with NaN values
    
    # Remove duplicates by summing quantity per item_name, date, and id_bakery
    df = df.groupby(['item_name', 'date', 'id_bakery']).agg({'quantity': 'sum'}).reset_index()
    
    # Set 'date' as the index
    df.set_index('date', inplace=True)
    
    # Further aggregation by item_name and date for daily data
    daily_data = df.groupby(['item_name']).resample('D').sum().reset_index(level=0, drop=True).fillna(0).reset_index()

    # print(daily_data.head())
    
    return daily_data

 # upfate push

def create_dataset(data, look_back=7):
    dataX, dataY = [], []
    for i in range(len(data) - look_back):
        dataX.append(data[i:(i + look_back)])
        dataY.append(data[i + look_back])
    return np.array(dataX), np.array(dataY)

def build_model(input_shape):
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=input_shape))
    model.add(LSTM(50))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [5]:

df = load_data_from_db()
df_processed = preprocess_data(df)

df.head()

,id_bakery,date,item_name,quantity
0,4,2021-01-02,BAGUETTE,1
1,85,2021-01-02,PAIN AU CHOCOLAT,3
2,85,2021-01-02,PAIN AU CHOCOLAT,2
3,84,2021-01-02,PAIN,1
4,138,2021-01-02,TRADITIONAL BAGUETTE,5


In [28]:
# look_back = 7
# models = {}

# df = load_data_from_db()
# df_processed = preprocess_data(df)

# for item_name in df_processed['item_name'].unique():
#     df_item = df_processed[df_processed['item_name'] == item_name].copy()
#     df_item = df_item[['quantity']]  # Use only 'quantity' for training
#     scaler = MinMaxScaler(feature_range=(0, 1))
#     df_item['quantity'] = scaler.fit_transform(df_item[['quantity']])
    
#     X, y = create_dataset(df_item['quantity'].values, look_back)
    
#     # Debugging: Print the shapes
#     print(f"Processing item: {item_name}")
#     print(f"Shape of X before reshape: {X.shape}")
#     print(f"Shape of y: {y.shape}")
    
#     if X.shape[0] == 0 or X.shape[1] == 0:
#         print(f"Skipping item: {item_name} due to insufficient data after preprocessing.")
#         continue
    
#     X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    
#     train_size = int(len(X) * 0.8)
#     X_train, X_test = X[:train_size], X[train_size:]
#     y_train, y_test = y[:train_size], y[train_size:]
    
#     model = build_model((look_back, 1))
    
#     model_path = 'models/all_items_best_model.keras'
#     checkpoint = ModelCheckpoint(model_path, monitor='val_loss', save_best_only=True, mode='min')
    
#     model.fit(X_train, y_train, epochs=20, batch_size=512, validation_data=(X_test, y_test), callbacks=[checkpoint])
    
#     models[item_name] = (model, scaler)

# df = load_data_from_db()
# df_processed = preprocess_data(df)

# look_back = 7
# models = {}

# # Train models for each item
# for item_name in df_processed['item_name'].unique():
#     df_item = df_processed[df_processed['item_name'] == item_name].copy()
#     df_item = df_item[['quantity']]  # Use only 'quantity' for training
#     scaler = MinMaxScaler(feature_range=(0, 1))
#     df_item['quantity'] = scaler.fit_transform(df_item[['quantity']])
    
#     X, y = create_dataset(df_item['quantity'].values, look_back)
    
#     # Debugging: Print the shapes
#     print(f"Processing item: {item_name}")
#     print(f"Shape of X before reshape: {X.shape}")
#     print(f"Shape of y: {y.shape}")
    
#     if X.shape[0] == 0 or X.shape[1] == 0:
#         print(f"Skipping item: {item_name} due to insufficient data after preprocessing.")
#         continue
    
#     X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    
#     train_size = int(len(X) * 0.8)
#     X_train, X_test = X[:train_size], X[train_size:]
#     y_train, y_test = y[:train_size], y[train_size:]
    
#     model = build_model((look_back, 1))
    
#     model_path = 'models/all_items_best_model.keras'
#     checkpoint = ModelCheckpoint(model_path, monitor='val_loss', save_best_only=True, mode='min')
    
#     model.fit(X_train, y_train, epochs=20, batch_size=512, validation_data=(X_test, y_test), callbacks=[checkpoint])
    
#     models[item_name] = (model, scaler)

df = load_data_from_db()
df_processed = preprocess_data(df)
    
look_back = 7
models = {}
    
for item_name in df_processed['item_name'].unique():
    df_item = df_processed[df_processed['item_name'] == item_name].copy()
    df_item = df_item[['quantity']]
    scaler = MinMaxScaler(feature_range=(0, 1))
    df_item['quantity'] = scaler.fit_transform(df_item[['quantity']])
    
    X, y = create_dataset(df_item['quantity'].values, look_back)
    if X.shape[0] == 0 or X.shape[1] == 0:
        # print(f"Skipping item: {item_name} due to insufficient data after preprocessing.")
        continue
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    
    train_size = int(len(X) * 0.8)
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]
    
    model = build_model((look_back, 1))
    model.fit(X_train, y_train, epochs=20, batch_size=512, validation_data=(X_test, y_test))
    
    models[item_name] = (model, scaler)

# Save all models and scalers to a single .pkl file
joblib.dump(models, 'models/models_dict.pkl')

        date item_name  id_bakery  quantity
0 2021-03-04         .        148         2
1 2021-03-05         0          0         0
2 2021-03-06         0          0         0
3 2021-03-07         0          0         0
4 2021-03-08         0          0         0
Skipping item: . due to insufficient data after preprocessing.


c:\Users\nuraf\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 5s 43ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 2/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 3/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 4/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 5/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 6/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 7/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 8/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 9/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 10/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 11/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.0000e+0

c:\Users\nuraf\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 0.1214 - val_loss: 0.0531
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - loss: 0.1035 - val_loss: 0.0431
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step - loss: 0.0866 - val_loss: 0.0340
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - loss: 0.0709 - val_loss: 0.0260
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - loss: 0.0565 - val_loss: 0.0192
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - loss: 0.0437 - val_loss: 0.0141
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - loss: 0.0331 - val_loss: 0.0109
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - loss: 0.0252 - val_loss: 0.0102
Epoch 9/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - loss: 0.0207 - val_loss: 0.0122
Epoch 10/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - loss: 0.0201 - val_loss: 0.0165
Epoch 11/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - loss: 0.0231 - val_loss: 0.0214
Epoch 12/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - loss: 0.0279 - val_loss: 0.0248
Epoch 13/20
1

...

c:\Users\nuraf\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 0.2190 - val_loss: 0.1756
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - loss: 0.1998 - val_loss: 0.1591
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - loss: 0.1816 - val_loss: 0.1433
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - loss: 0.1640 - val_loss: 0.1283
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - loss: 0.1471 - val_loss: 0.1138
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 0.1308 - val_loss: 0.1000
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - loss: 0.1150 - val_loss: 0.0869
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - loss: 0.0998 - val_loss: 0.0747
Epoch 9/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - loss: 0.0854 - val_loss: 0.0636
Epoch 10/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - loss: 0.0720 - val_loss: 0.0542
Epoch 11/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - loss: 0.0600 - val_loss: 0.0469
Epoch 12/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - loss: 0.0500 - val_loss: 0.0426
Epoch 13/20
1/

c:\Users\nuraf\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 0.0646 - val_loss: 0.0653
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/step - loss: 0.0545 - val_loss: 0.0584
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - loss: 0.0455 - val_loss: 0.0521
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - loss: 0.0373 - val_loss: 0.0465
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step - loss: 0.0300 - val_loss: 0.0417
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - loss: 0.0235 - val_loss: 0.0375
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step - loss: 0.0180 - val_loss: 0.0343
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step - loss: 0.0136 - val_loss: 0.0320
Epoch 9/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step - loss: 0.0104 - val_loss: 0.0309
Epoch 10/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - loss: 0.0086 - val_loss: 0.0311
Epoch 11/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step - loss: 0.0085 - val_loss: 0.0323
Epoch 12/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - loss: 0.0097 - val_loss: 0.0340
Epoch 13/

['models/models_dict.pkl']

In [8]:
# def predict_sales(model, data, scaler, look_back):
#     dataX, _ = create_dataset(data, look_back)
#     dataX = np.reshape(dataX, (dataX.shape[0], look_back, 1))
    
#     predictions = model.predict(dataX)
    
#     # Reverse scaling if needed
#     if scaler:
#         predictions = scaler.inverse_transform(predictions)
    
#     return predictions

def predict_sales(model, data, scaler, look_back, prediction_days):
    dataX, _ = create_dataset(data, look_back)
    dataX = np.reshape(dataX, (dataX.shape[0], look_back, 1))
    
    predictions = []
    input_sequence = data[-look_back:]
    
    for _ in range(prediction_days):
        input_sequence = np.reshape(input_sequence, (1, look_back, 1))
        next_prediction = model.predict(input_sequence)[0][0]
        predictions.append(next_prediction)
        input_sequence = np.append(input_sequence[:, 1:, :], [[[next_prediction]]], axis=1)
    
    # Reverse scaling if needed
    if scaler:
        predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))
    
    return predictions

In [70]:
# item_name_code = 'CROISSANT'

# if not os.path.exists('models/all_items_best_model.keras'):
#     print("error: Model gabungan tidak ditemukan.")
# else:
#     model = load_model('models/all_items_best_model.keras')
    
#     df_item = df_processed[df_processed['item_name'] == item_name_code].copy()
#     df_item = df_item[['quantity']]
    
#     # Check if there is enough data for prediction
#     if len(df_item) < look_back:
#         print(f"Insufficient data for item: {item_name_code}")
#     else:
#         scaler = models[item_name_code][1]  # Load scaler for the specific item
#         df_item['quantity'] = scaler.transform(df_item[['quantity']])
        
#         predictions = predict_sales(model, df_item['quantity'].values, scaler, look_back)
#         print(predictions)

# Predict sales for the next 7 days for CROISSANT
from datetime import timedelta
item_name_code = 'CROISSANT'
prediction_days = 7

if not os.path.exists('models/models_dict.pkl'):
    print('error: Models file not found'),
    
models = joblib.load('models/models_dict.pkl')
    
if item_name_code in models:
    model, scaler = models[item_name_code]
    df = load_data_from_db()
    df_processed = preprocess_data(df)
        
    df_item = df_processed[df_processed['item_name'] == item_name_code].copy()
    df_date = df_item[['date']]
    df_item = df_item[['quantity']]
        
    if len(df_item) < 7:
        print(f"Insufficient data for item: {item_name_code}")
        
    df_item['quantity'] = scaler.transform(df_item[['quantity']])
        
    predictions = predict_sales(model, df_item['quantity'].values, scaler, 7, prediction_days)
    
    last_date = pd.to_datetime(df_date.iloc[-1].date)
    
    future_dates = [last_date + timedelta(days=i) for i in range(1, len(predictions) + 1)]

    # Buat DataFrame untuk menggabungkan tanggal dengan hasil prediksi
    df_predictions = pd.DataFrame({'date': future_dates, 'predicted_quantity': predictions.flatten()})

    print(f"{df_predictions}")
    print(f"{df_date}")
    

# if item_name_code in models:
#     model, scaler = models[item_name_code]
#     df_item = df_processed[df_processed['item_name'] == item_name_code].copy()
#     df_item = df_item[['quantity']]
    
#     if len(df_item) < look_back:
#         print(f"Insufficient data for item: {item_name_code}")
#     else:
#         df_item['quantity'] = scaler.transform(df_item[['quantity']])
        
#         predictions = predict_sales(model, df_item['quantity'].values, scaler, look_back, prediction_days)
#         print(f"Predictions for the next {prediction_days} days: {predictions}")
# else:
#     print(f"Model for item {item_name_code} not found.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 242ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
        date  predicted_quantity
0 2022-01-01           53.692993
1 2022-01-02           50.655533
2 2022-01-03           48.649052
3 2022-01-04           50.977852
4 2022-01-05           52.387493
5 2022-01-06           55.040401
6 2022-01-07           57.418941
           date
7346 2021-01-02
7347 2021-01-03
7348 2021-01-04
7349 2021-01-05
7351 2021-01-07
...         ...
7705 2021-12-27
7706 2021-12-28
7707 2021-12-29
7708 2021-12-30
7709 2021-12-31

[338 rows x 1 columns]


In [67]:
from datetime import timedelta

predictions = np.array([[53.692993], [50.655533], [48.64905], [50.97785], [52.387493], [55.0404], [57.41894]])

# Ambil tanggal terakhir dari data Anda
# Misalnya, tanggal terakhir adalah 2024-08-01
last_date = pd.to_datetime('2024-08-01')

# Buat rangkaian tanggal mulai dari hari berikutnya hingga 7 hari ke depan
future_dates = [last_date + timedelta(days=i) for i in range(1, len(predictions) + 1)]

# Buat DataFrame untuk menggabungkan tanggal dengan hasil prediksi
df_predictions = pd.DataFrame({'date': future_dates, 'predicted_quantity': predictions.flatten()})

print(future_dates)

[Timestamp('2024-08-02 00:00:00'), Timestamp('2024-08-03 00:00:00'), Timestamp('2024-08-04 00:00:00'), Timestamp('2024-08-05 00:00:00'), Timestamp('2024-08-06 00:00:00'), Timestamp('2024-08-07 00:00:00'), Timestamp('2024-08-08 00:00:00')]


In [34]:
data = load_data_from_db()
data.tail()

,id_bakery,date,item_name,quantity
131415,4,2021-12-31,BAGUETTE,1
131416,49,2021-12-31,FICELLE,1
131417,4,2021-12-31,BAGUETTE,1
131418,84,2021-12-31,PAIN,1
131419,33,2021-12-31,CROISSANT AMANDES,1
